In [1]:
# Multilayer Perceptron (MLP) for multi-class softmax classification:
# modified from 
# https://keras.io/getting-started/sequential-model-guide/#multilayer-perceptron-mlp-for-multi-class-softmax-classification

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam

import random

C:\Users\ortci\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import numpy as np

n_samples = 1000000
n_partitions = 10

# Generate tasks and partitions
partition_data = np.random.random((n_samples, n_partitions)) # partition data generation

#X = np.zeros((n_samples, n_partitions + 1))             # initialize input layer
y = np.zeros((n_samples, n_partitions))                 # initialize outputs layer for training 

task_data = np.zeros((n_samples, 1))                         # initialize task list

for i in range (0, n_samples):
    
    partitions = partition_data[i]
    task = random.uniform(0, partitions.max())
    task_data[i] = task
    
    for j in range (0, n_partitions):
        current_fit = partitions[j] - task
        if current_fit < 0:
            current_fit = -1
        y[i,j] = current_fit
            
X = np.hstack((task_data,partition_data))

In [3]:
# split data between train and test set 
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [4]:
import tensorflow as tf

model = Sequential()
n_hidden_units = 1000
# Dense(n_hidden_units) is a fully-connected layer with n_hidden_units hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 10-dimensional vectors.
model.add(Dense(n_hidden_units, activation='elu', input_dim=n_partitions+1))
#model.add(Dropout(0.5))
model.add(Dense(10, activation='elu'))

# optimizer options
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
rmsprop = tf.train.RMSPropOptimizer(0.008)
adam = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

In [5]:
model.compile(loss='mean_squared_error',
              optimizer=sgd,
              metrics=['mae'])

In [6]:
batchsize = 500

model.fit(X_train, y_train,
          epochs=10,
          batch_size=batchsize)
score = model.evaluate(X_test, y_test, batch_size=batchsize)
print(score)

'''
Why is the training loss much higher than the testing loss?

A Keras model has two modes: training and testing. Regularization mechanisms, such as Dropout and 
L1/L2 weight regularization, are turned off at testing time. Besides, the training loss is the average 
of the losses over each batch of training data. Because your model is changing over time, the loss over 
the first batches of an epoch is generally higher than over the last batches. On the other hand, the 
testing loss for an epoch is computed using the model as it is at the end of the epoch, resulting in a lower loss.
'''

Epoch 1/10
800000/800000 [==============================] - 21s 27us/step - loss: 0.1333 - mean_absolute_error: 0.3228
Epoch 2/10
800000/800000 [==============================] - 21s 27us/step - loss: 0.1025 - mean_absolute_error: 0.2880
Epoch 3/10
800000/800000 [==============================] - 21s 27us/step - loss: 0.1023 - mean_absolute_error: 0.2877
Epoch 4/10
800000/800000 [==============================] - 22s 27us/step - loss: 0.1021 - mean_absolute_error: 0.2874
Epoch 5/10
800000/800000 [==============================] - 22s 28us/step - loss: 0.1020 - mean_absolute_error: 0.2870
Epoch 6/10
800000/800000 [==============================] - 22s 27us/step - loss: 0.1018 - mean_absolute_error: 0.2867
Epoch 7/10
800000/800000 [==============================] - 21s 26us/step - loss: 0.1016 - mean_absolute_error: 0.2864
Epoch 8/10
800000/800000 [==============================] - 21s 26us/step - loss: 0.1014 - mean_absolute_error: 0.2860
Epoch 9/10
800000/800000 [======================

'\nWhy is the training loss much higher than the testing loss?\n\nA Keras model has two modes: training and testing. Regularization mechanisms, such as Dropout and \nL1/L2 weight regularization, are turned off at testing time. Besides, the training loss is the average \nof the losses over each batch of training data. Because your model is changing over time, the loss over \nthe first batches of an epoch is generally higher than over the last batches. On the other hand, the \ntesting loss for an epoch is computed using the model as it is at the end of the epoch, resulting in a lower loss.\n'

In [7]:
from keras.models import load_model

model.save('MLP_elu_softmax_11_inputs_v4.h5')  # creates a HDF5 file 'my_model.h5'
model.save_weights('MLP_elu_softmax_11_inputs_weights_v4.h5')

'''
del model  # deletes the existing model

# returns a compiled model
# identical to the previous one
model = load_model('MLP_Multiclass_softmax_10_inputs.h5')
model.load_weights('MLP_Multiclass_softmax_10_inputs_weights.h5') # for same architecture
model.load_weights('MLP_Multiclass_softmax_10_inputs_weights.h5', by_name=True) # for different architecture
'''

"\ndel model  # deletes the existing model\n\n# returns a compiled model\n# identical to the previous one\nmodel = load_model('MLP_Multiclass_softmax_10_inputs.h5')\nmodel.load_weights('MLP_Multiclass_softmax_10_inputs_weights.h5') # for same architecture\nmodel.load_weights('MLP_Multiclass_softmax_10_inputs_weights.h5', by_name=True) # for different architecture\n"

In [8]:
y_pred = model.predict(X_test, batch_size=batchsize)

In [9]:
samp_no = 18
print(y_pred[samp_no])
print(y_test[samp_no])
print(X_test[samp_no])

[ 0.5918642  -0.33407295  0.6789075  -0.35013735  0.6506874  -0.3998338
  0.00395453 -0.578178   -0.20163131  0.6632808 ]
[ 0.52385518  0.08315324  0.56288971  0.07167284  0.54512781  0.02940142
  0.26578348 -1.          0.1596448   0.55138711]
[0.35775022 0.8816054  0.44090346 0.92063993 0.42942306 0.90287803
 0.38715164 0.6235337  0.23351293 0.51739502 0.90913734]
